In [3]:
import os
import pandas as pd
import numpy as np

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

## Generate cleaned fasta for first round of tree drawing

In [ ]:
cols = ['sseqid', 'evalue', 'bitscore', 'pident', 'length', 'staxid', 'scomnames', 'ssciname', 'sblastnames', 'sseq']

a = pd.read_csv("../23may2023/bspaONLY_blastn_out_subj_besthit_23may2023.tsv", delimiter="\t", names=cols)
b = pd.read_csv("../23may2023/bspaONLY_blastn_out_taxid_limited_23may2023.tsv", delimiter="\t", names=cols)

df = pd.concat([a, b])
df.shape

In [ ]:
df = df[df['length'] > 80].reset_index(drop=True)

In [ ]:
df['genus'] = df['ssciname'].str.split(" ", expand=True)[0]

In [ ]:
# df[['staxid', 'genus']].to_csv("23may2023/blastn_genus_names.csv", index=False)

In [ ]:
# df['scomnames'].unique()

drop = ['Mycobacterium malmoense',
 'Mycobacterium frederiksbergens',
 'Mycolicibacterium vinylchloridicum',
 'Mycobacterium seoulense',
 'Mycobacterium asiaticum DSM 44297',
 'Mycobacterium simulans',
 'Mycolicibacterium anyangense',
 'Mycobacterium cookii',
 'Mycolicibacterium hippocampi',
 'Mycobacterium conspicuum'
]

In [ ]:
df = df[~df['scomnames'].isin(drop)]

In [ ]:
df = df.sort_values(by='length', ascending=False) 
df = df[~df['staxid'].duplicated()]

In [ ]:
df.shape

In [ ]:
df['length'].describe()

In [ ]:
# df = df[df['length'] > 100].reset_index(drop=True)

In [81]:
sequences = []

for i in df.index:
    sequences.append(SeqRecord(Seq(df.loc[i, 'sseq']), id=df.loc[i,'staxid'].astype(str), description=""))


In [82]:
with open("../blastn_bspa_hits_100bp_19Jul2023.fasta", "w") as output_handle:
    SeqIO.write(sequences, output_handle, "fasta")

## Get names from taxid values for 100 species remaining after treemmer round 1

In [103]:
# Treemmer 100n subset list read in, and some manual alterations made. Now reading in those manually selected species to pull all desired taxid values

# sp = pd.read_csv("blastn_bsap_cleaned_hits_round1_consensus.nwk_trimmed_list_X_100", header=None)
sp = pd.read_excel("species_to_keep_subset100.xlsx", header=None)

In [145]:
df[df['scomnames'].isin(sp[0])].reset_index(drop=True)['length'].describe()

count     98.000000
mean     124.836735
std       42.637695
min       81.000000
25%       93.000000
50%      110.000000
75%      142.500000
max      238.000000
Name: length, dtype: float64

In [146]:
df[df['scomnames'].isin(sp[0])].reset_index(drop=True)['pident'].describe()

count    98.000000
mean     76.059255
std       3.335523
min      70.000000
25%      73.742500
50%      75.791000
75%      78.617500
max      83.871000
Name: pident, dtype: float64

In [148]:
df[df['scomnames'].isin(sp[0])].reset_index(drop=True).to_csv("species_to_keep_subset100.csv", index=False)

In [153]:
",".join(df[df['scomnames'].isin(sp[0])]['staxid'].astype(str).tolist())

'2738409,1273442,564198,1908205,1801,1841861,120959,1809,1779,1260918,350058,182220,1460372,1772,84962,1769,443218,2742202,36811,1249101,126673,1053547,589162,1788,1073574,1223544,158898,1210074,2487352,391736,526226,1990687,191292,543736,2831662,912594,1755582,43767,521096,1348662,2760083,1219361,558173,2895283,1206730,2604572,885042,300028,683150,1848329,1200352,2592067,1437874,1210084,2320431,2762214,1219029,1089545,1048339,479432,37330,108486,571913,1220553,364929,2545757,455432,2911517,543632,1068980,1121368,225051,1572660,83302,547056,664692,2750812,452863,630975,675635,1993,2508997,485602,1654476,35755,479435,280293,1431546,931089,1045776,1105145,2576905,106370,1677852,1128676,1686286,2761101,1562887'

In [ ]:
## used these coordinates to run blast again to fetch start and end coords for hits for each of these species for
## bioB and bsaP

In [191]:
cols = ['sseqid', 'evalue', 'bitscore', 'pident', 'length', 'staxid', 'scomnames', 'ssciname', 'sstart', 'ssend', 'sseq']

biob = pd.read_csv("biob_subset100_blastn_out_taxid_limited_19Jul2023.tsv", delimiter="\t", names=cols)
bsap = pd.read_csv("bsap_subset100_blastn_out_taxid_limited_19Jul2023.tsv", delimiter="\t", names=cols)

In [175]:
biob = biob.sort_values(by='bitscore', ascending=False).drop_duplicates(subset='staxid').reset_index(drop=True)
bsap = bsap.sort_values(by='bitscore', ascending=False).drop_duplicates(subset='staxid').reset_index(drop=True)

In [179]:
biob.head()

,sseqid,evalue,bitscore,pident,length,staxid,scomnames,ssciname,sstart,ssend,sseq
0,gi|2171054877|ref|NZ_CP089224.1|,0.0,1265.0,86.415,1060,2738409,Mycobacterium ostraviense,Mycobacterium ostraviense,2729619,2728560,GTGACTCAAGCGCCGATTCGACCGACGACCGACGCCGGCCAGCAGG...
1,gi|1143091011|ref|NZ_FUEZ01000004.1|,0.0,1233.0,86.000,1050,1841861,Mycobacterium numidiamassiliense,Mycobacterium numidiamassiliense,3784479,3783442,GTGACTCAAGCGGCAACTCGACCGACAACCGAAGCCGGC------G...
2,gi|1184639431|ref|NZ_LQOW01000034.1|,0.0,1213.0,86.961,997,1260918,Mycobacterium fragae,Mycobacterium fragae,140783,141779,ACAGCACCGACATCCTTGCCGTGGCGCGGCAACAGGTGCTCGACCG...
3,gi|2203294655|ref|NZ_CP085200.1|,0.0,1207.0,85.510,1049,1809,Mycobacterium ulcerans,Mycobacterium ulcerans,3972206,3971158,GTGACTCAGGCGGCTACTCGACCCAGCAATGACGCCGGCCAGGATG...
4,gi|1895562399|ref|NZ_CP045081.1|,0.0,1207.0,85.510,1049,120959,Mycobacterium kubicae,Mycobacterium kubicae,2385368,2386416,GTGACTCAAGCGCCGATTCGACCGACCACCGACGCCGGCCAGGATG...


In [185]:
subset100 = bsap[['sseqid', 'staxid', 'ssciname', 'pident', 'length', 'sstart', 'ssend']].merge(
    biob[['sseqid', 'pident', 'length', 'sstart', 'ssend']], on='sseqid', how='outer').sort_values(by='sseqid')

In [186]:
subset100.columns=['sseqid', 'staxid', 'ssciname', 
                   'bspa_pident','bspa_len','bspa_start','bspa_end',
                   'biob_pident','biob_len','biob_start','biob_end']

In [188]:
subset100.to_csv("subset100_feature_coords.csv", index=None)

In [ ]:
bspa

In [194]:
biob[biob['staxid'] == 1219361]

,sseqid,evalue,bitscore,pident,length,staxid,scomnames,ssciname,sstart,ssend,sseq
205,gi|1055243950|ref|NZ_BCRN01000023.1|,0.000,850.0,81.167,908,1219361,Millisia brevis NBRC 105863,Millisia brevis NBRC 105863,227,1131,TGCTCGAGGTGTTGACGTTGCCCGACGAGCGCATCGAGGAGCTGCT...
401,gi|1055244024|ref|NZ_BCRN01000033.1|,0.024,44.6,88.235,34,1219361,Millisia brevis NBRC 105863,Millisia brevis NBRC 105863,11641,11674,GCGCACCATGCCGCGGTTCGCCGGGCGACGCGAG


## Pull sequences for bioB and bsaP for subset100 seqs

In [106]:
# df = pd.read_csv("subset78_feature_loci.csv")

df = pd.read_excel("subset100_feature_coords.xlsx")


In [44]:
# df.dropna(subset=['staxid'])['staxid'].astype(int).to_csv('subset80_taxids.txt', index=False)

In [109]:
df['bsap_seq'] = df['bsap_seq'].str.replace(" ", "")
df['biob_seq'] = df['biob_seq'].str.replace(" ", "")

In [64]:
bsap_nt_seqs = []
bsap_aa_seqs = []
biob_aa_seqs = []

for i in df[['ssciname', 'bsap_seq']].dropna().index:
    bsap_nt_seqs.append(SeqRecord(Seq(df.loc[i, 'bsap_seq']), df.loc[i, 'ssciname'].replace(" ", "_"), description=""))
    bsap_aa_seqs.append(SeqRecord(Seq(df.loc[i, 'bsap_seq']).translate(table=11), df.loc[i, 'ssciname'].replace(" ", "_"), description=""))
    
    biob_aa_seqs.append(SeqRecord(Seq(df.loc[i, 'biob_seq']).translate(table=11), df.loc[i, 'ssciname'].replace(" ", "_"), description=""))

In [67]:
# SeqIO.write(bsap_nt_seqs, "bsap_nt_sequences_subset80.fa", "fasta")
# SeqIO.write(bsap_aa_seqs, "bsap_protein_sequences_subset80.fa", "fasta")
# SeqIO.write(biob_aa_seqs, "biob_protein_sequences_subset80.fa", "fasta")

77

#### Added psuedocardia sp and H37Rv entries to each above fasta manually!

H37Rv entries pulled from mycobrowser

pseudocardia aa seq (directly from Inna):
MLAVLQLPDDRVDELLALAHEVRMRWCGPEVEVEGIVSVKTGGCPEDCHFCSQSGHFDSPVRAVWLNIAGLVEAAKQTAATGATEFCIVAAVRGPDRRLMSQVKAGIEAIRKEPLTAHLNIACSLGMLTSEQVDELAAMGVHRYNHNLETARSYFPTVVTTHTWEERFQTLRMVREAGMEVCCGGIIGLGETVAQRAEFAAQLASLDPDEVPMNFLIPQPGTPYEDYDVVAGAEALKTVAAFRFALPRTILRFAGGRELTLGDLGAERGMLGGINAIIVGNYLTNLGRPAQADLDMLTDLRMPIKALGQTFCDVCGRPAGEGNHASCARRRGLEPPRYCPQCARRMVVQVMPAGWSARCSRHGVQPGLG

pseudocardia nt seq (https://www.bv-brc.org/view/FeatureList/?keyword%28%22LC644_01795%22%29=#view_tab=features&defaultSort=-score):
>fig|60912.15.peg.2225|LC644_01795| Biotin synthase (EC 2.8.1.6) [Pseudonocardia sp. MAG106 | 60912.15]
gtgacccggtcgcaggtcgatgtgctggccgtcgcccgacaacgggtactccaccaggga
gcagggctgtctgagcagcaggtgctcgctgtacttcaactgccggacgaccgggtggat
gagctgctcgcgctggcccacgaggtgcggatgcgctggtgcggcccggaggtcgaagtc
gagggcatcgtcagcgtcaagaccggcggctgcccggaggattgtcacttctgctcgcag
tcgggccacttcgactcaccggtacgggcggtgtggctgaacattgccggcctagtcgag
gcagccaagcagactgcggccaccggggcgaccgagttctgcatcgtcgccgccgtgcgc
ggaccggaccggcggttgatgtcgcaggtcaaagccggcatcgaggcgatccgcaaagaa
cccttgaccgcgcatctcaacatcgcctgctcgctgggcatgctcacctcagaacaagtg
gacgagctggccgccatgggcgtgcaccgctacaaccacaacttggagaccgcgcgctcg
tacttcccgacggtggtgaccacccacacctgggaggagcgtttccagacgctgcggatg
gttcgcgaggccgggatggaagtctgctgcggcggcatcatcgggctgggggagacggtg
gcccagcgcgccgagttcgccgcgcagctcgcctcgctggatcccgacgaggtgccgatg
aacttcctgatcccgcagcccggcaccccgtacgaggactatgacgtggtggccggtgcc
gaggcgctcaagaccgtcgcggcgttccggttcgcgctgccccgcacaatcctgcgcttc
gccgggggccgtgagctcaccctgggtgatctgggcgccgagcgaggcatgctgggcggc
atcaatgcgatcatcgtgggcaactacctgaccaatctcggtcgaccggcgcaggcggat
ctggacatgctcaccgacctgcggatgccgatcaaggcgctgggccagaccttctgcgat
gtctgcgggcgcccggccggcgaaggaaaccatgcgagctgcgcccgccgccgcggcctg
gagccgccccggtactgcccgcaatgtgcgcgccggatggtggtgcaggtaatgccggcc
ggctggtcggcgcggtgcagcaggcacggtgtgcagccggggttgggatga


For BsaP Pseudonocardia entries, the values entered are subsets of bioB, with length 1.5x the length of H37Rv BsaP

In [69]:
pseudonocardia_aa = "MLAVLQLPDDRVDELLALAHEVRMRWCGPEVEVEGIVSVKTGGCPEDCHFCSQSGHFDSPVRAVWLNIAGLVEAAKQTAATGATEFCIVAAVRGPDRRLMSQVKAGIEAIRKEPLTAHLNIACSLGMLTSEQVDELAAMGVHRYNHNLETARSYFPTVVTTHTWEERFQTLRMVREAGMEVCCGGIIGLGETVAQRAEFAAQLASLDPDEVPMNFLIPQPGTPYEDYDVVAGAEALKTVAAFRFALPRTILRFAGGRELTLGDLGAERGMLGGINAIIVGNYLTNLGRPAQADLDMLTDLRMPIKALGQTFCDVCGRPAGEGNHASCARRRGLEPPRYCPQCARRMVVQVMPAGWSARCSRHGVQPGLG"
pseudonocardia_nt = "gtgacccggtcgcaggtcgatgtgctggccgtcgcccgacaacgggtactccaccagggagcagggctgtctgagcagcaggtgctcgctgtacttcaactgccggacgaccgggtggatgagctgctcgcgctggcccacgaggtgcggatgcgctggtgcggcccggaggtcgaagtcgagggcatcgtcagcgtcaagaccggcggctgcccggaggattgtcacttctgctcgcagtcgggccacttcgactcaccggtacgggcggtgtggctgaacattgccggcctagtcgaggcagccaagcagactgcggccaccggggcgaccgagttctgcatcgtcgccgccgtgcgcggaccggaccggcggttgatgtcgcaggtcaaagccggcatcgaggcgatccgcaaagaacccttgaccgcgcatctcaacatcgcctgctcgctgggcatgctcacctcagaacaagtggacgagctggccgccatgggcgtgcaccgctacaaccacaacttggagaccgcgcgctcgtacttcccgacggtggtgaccacccacacctgggaggagcgtttccagacgctgcggatggttcgcgaggccgggatggaagtctgctgcggcggcatcatcgggctgggggagacggtggcccagcgcgccgagttcgccgcgcagctcgcctcgctggatcccgacgaggtgccgatgaacttcctgatcccgcagcccggcaccccgtacgaggactatgacgtggtggccggtgccgaggcgctcaagaccgtcgcggcgttccggttcgcgctgccccgcacaatcctgcgcttcgccgggggccgtgagctcaccctgggtgatctgggcgccgagcgaggcatgctgggcggcatcaatgcgatcatcgtgggcaactacctgaccaatctcggtcgaccggcgcaggcggatctggacatgctcaccgacctgcggatgccgatcaaggcgctgggccagaccttctgcgatgtctgcgggcgcccggccggcgaaggaaaccatgcgagctgcgcccgccgccgcggcctggagccgccccggtactgcccgcaatgtgcgcgccggatggtggtgcaggtaatgccggccggctggtcggcgcggtgcagcaggcacggtgtgcagccggggttgggatga"

In [80]:
pseudonocardia_nt[-360:]

'ctgcgcttcgccgggggccgtgagctcaccctgggtgatctgggcgccgagcgaggcatgctgggcggcatcaatgcgatcatcgtgggcaactacctgaccaatctcggtcgaccggcgcaggcggatctggacatgctcaccgacctgcggatgccgatcaaggcgctgggccagaccttctgcgatgtctgcgggcgcccggccggcgaaggaaaccatgcgagctgcgcccgccgccgcggcctggagccgccccggtactgcccgcaatgtgcgcgccggatggtggtgcaggtaatgccggccggctggtcggcgcggtgcagcaggcacggtgtgcagccggggttgggatga'

### Write species and genus file from subset100 to file for Treemmer

In [110]:
df['genus'] = df['ssciname'].str.split(expand=True)[0]
df['name'] = df['ssciname'].str.replace(" ", "_")

In [113]:
df[['name', 'genus', 'bsap_seq']].dropna()[['name', 'genus']].to_csv("subset80_genus_names_lm.csv", index=False)

## Write subset 50 fasta file from subset50 Treemmer round:

In [130]:
subset = pd.read_csv("blastn_bsap_cleaned_hits_round2_consensus.nwk_trimmed_list_X_50", header=None)

In [131]:
df[df['name'].isin(subset[0])].shape

(50, 23)

In [132]:
bsap_nt_seqs = []
bsap_aa_seqs = []
biob_aa_seqs = []

for i in df[df['name'].isin(subset[0])].index:
    bsap_nt_seqs.append(SeqRecord(Seq(df.loc[i, 'bsap_seq']), df.loc[i, 'ssciname'].replace(" ", "_"), description=""))
    bsap_aa_seqs.append(SeqRecord(Seq(df.loc[i, 'bsap_seq']).translate(table=11), df.loc[i, 'ssciname'].replace(" ", "_"), description=""))
    
    biob_aa_seqs.append(SeqRecord(Seq(df.loc[i, 'biob_seq']).translate(table=11), df.loc[i, 'ssciname'].replace(" ", "_"), description=""))

In [133]:
SeqIO.write(bsap_nt_seqs, "bsap_nt_sequences_subset50.fa", "fasta")
SeqIO.write(bsap_aa_seqs, "bsap_protein_sequences_subset50.fa", "fasta")
SeqIO.write(biob_aa_seqs, "biob_protein_sequences_subset50.fa", "fasta")

50